# Churn Prediction Using Classification

In here, I am going to use two methods, using the actual samples and using resampling method to make each class have equal samples. The expectation when using resampling method is to avoid bias because when testing the models with actual samples, they tend to predict all samples to not churn (because the data contain more `0` class).

# Building Models

In [13]:
# importing libraries
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy.stats import pearsonr, chi2_contingency
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.feature_selection import RFE
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
%matplotlib inline

In [4]:
data = pd.read_csv('PrepChurnData.csv')
data.head()

,CustomerId,Surname,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Complain,Satisfaction Score,Card Type,Point Earned,Female,Male,France,Germany,Spain
0,15634602,Hargrave,619,42,2,0.00,1,1,1,101348.88,1,1,2,DIAMOND,464,1,0,1,0,0
1,15647311,Hill,608,41,1,83807.86,1,0,1,112542.58,0,1,3,DIAMOND,456,1,0,0,0,1
2,15619304,Onio,502,42,8,159660.80,3,1,0,113931.57,1,1,3,DIAMOND,377,1,0,1,0,0
3,15701354,Boni,699,39,1,0.00,2,0,0,93826.63,0,0,5,GOLD,350,1,0,1,0,0
4,15737888,Mitchell,850,43,2,125510.82,1,1,1,79084.10,0,0,5,GOLD,425,1,0,0,0,1


In [5]:
features = ['CreditScore','Age','Tenure','Balance','NumOfProducts','HasCrCard','IsActiveMember','EstimatedSalary',
            'Satisfaction Score','Point Earned','Female','Male','France','Germany','Spain']
target = 'Exited'

In [6]:
train, test = train_test_split(data, test_size=0.2,random_state=42)

In [7]:
X_train = train[features]
y_train = train[target].astype(str)
X_test = test[features]
y_test = test[target].astype(str)

In [8]:
print('Number of training samples:', X_train.shape[0])
print('Number of test samples:', X_test.shape[0])

Number of training samples: 8000
Number of test samples: 2000


## Logistic Regression

In [9]:
Logreg = LogisticRegression()
Logreg.fit(X_train, y_train)

LogisticRegression()

In [10]:
y_hatlr = Logreg.predict(X_test)

In [12]:
accuracy_score(y_hatlr, y_test)

0.7995

In [ ]:
pipe = make_pipeline(StandardScaler(), LogisticRegression())
pipe.fit(X_train,y_train)